In [1]:
from PIL import Image
import numpy as np
import pandas as pd
import pandas_datareader as pdr
import os

In [2]:
import time
import progressbar
import talib
from tqdm.notebook import tqdm

In [3]:
# change height

def img_graph(input_data, height = 64):
    
    data = input_data.copy()
    
    lenght = 3 * len(data)
    
    im = Image.new(mode = '1', size = (lenght, height))
    
    maximum = data.loc[:, ['OPENPRC', 'ASKHI', 'BIDLO', 'PRC', 'MA']].max().max()
    minimum = data.loc[:, ['OPENPRC', 'ASKHI', 'BIDLO', 'PRC', 'MA']].min().min()
    
    price_height = height # - volume_height
    data.loc[:, ['OPENPRC', 'ASKHI', 'BIDLO', 'PRC', 'MA']] = round(((data.loc[:, ['OPENPRC', 'ASKHI', 'BIDLO', 'PRC', 'MA']] - minimum)/(maximum - minimum)) * price_height)
    data[data <= 0] = 1
    
    matrix = pd.DataFrame(np.array(im))

    for i,j in zip(range(1, lenght - 1, 3), data.index):
        
        try:
            o = price_height - int(data.loc[j,'OPENPRC'])
            h = price_height - int(data.loc[j,'ASKHI'])
            l = price_height - int(data.loc[j,'BIDLO'])
            c = price_height - int(data.loc[j,'PRC'])
            ma = price_height - int(data.loc[j, 'MA'])
            if o < 0 or h < 0 or l < 0 or c < 0:            
                matrix.iloc[v:, int(i)] = True
            else:
                matrix.iloc[int(o),int(i) - 1] = True
                matrix.iloc[int(h):int(l) + 1, int(i)] = True
                matrix.iloc[int(c), int(i) + 1] = True
                matrix.iloc[int(ma), int(i) - 1 : int(i) + 1] = True
        except: pass

    graph = Image.fromarray(np.array(matrix).astype(int))
#     graph.show()
    return(graph)


In [4]:
import pyarrow.feather as feather
# meta = pd.read_excel('Ameria/EUR.xlsx', index_col = 0, engine = "openpyxl")
meta = feather.read_feather(r"C:\Users\hayk.darbinyan\Downloads\complete_data.feather")
print(len(meta))
# meta.dropna(axis = 0, inplace = True)
# print(len(meta))
meta.head()

73691132


,date,OPENPRC,ASKHI,BIDLO,PRC,VOL,permno
0,2008-08-31 17:00:00,9.4634,9.4634,9.4634,9.4634,0,eursek
1,2008-08-31 17:08:00,9.4636,9.4636,9.4636,9.4636,0,eursek
2,2008-09-01 16:37:00,9.4604,9.4604,9.4603,9.4603,0,eursek
3,2008-09-01 17:24:00,9.4627,9.4627,9.4627,9.4627,0,eursek
4,2008-09-01 17:25:00,9.4603,9.4627,9.4603,9.4627,0,eursek


In [5]:
meta = meta[meta['permno'] == 'eurusd']
meta = meta.iloc[100000:]

In [6]:
meta.columns = ['date', 'OPENPRC', 'ASKHI', 'BIDLO', 'PRC', 'VOL', 'Permno']
meta.set_index('Permno', inplace = True)
meta.head()

,date,OPENPRC,ASKHI,BIDLO,PRC,VOL
Permno,,,,,,
eurusd,2000-11-01 03:28:00,0.8539,0.8539,0.8539,0.8539,0
eurusd,2000-11-01 03:29:00,0.8538,0.8538,0.8538,0.8538,0
eurusd,2000-11-01 03:30:00,0.8539,0.8541,0.8539,0.8541,0
eurusd,2000-11-01 03:31:00,0.8546,0.8546,0.8544,0.8544,0
eurusd,2000-11-01 03:32:00,0.8543,0.8543,0.8542,0.8542,0


In [7]:
pair = 'EURUSD'
observation_period = 20
height = 64
test_period = 10
subset = 'All'
iters = [pair]
indicator = ''
a_count = 0
labels_df = pd.DataFrame(columns = ['Date', 'Permno', 'Ret_5d', 'Ret_20d', 'Ret_60d'])

pbar = progressbar.ProgressBar(maxval = len(iters)).start()

for k, permno in enumerate(list(iters)):
    
    a_count = 0
    labels_df = pd.DataFrame(columns = ['Date', 'Permno', 'Ret_5d', 'Ret_20d', 'Ret_60d'])
    
    data = meta.loc[permno, :]
    data = data.loc[:, ['date', 'OPENPRC', 'ASKHI', 'BIDLO', 'PRC', 'VOL']]
    data['MA'] = data['PRC'].rolling(observation_period).mean()
    data = data.iloc[observation_period + 1:, :]
    data['ret_5d'] = np.zeros(len(data))
    data['ret_20d'] = np.zeros(len(data))
    data['ret_60d'] = np.zeros(len(data))
    data.set_index('date', inplace = True)
    data = data.iloc[-1600000:]
    l = len(data)
    iterations = round(l/(test_period)) - 5
    
    pic_path = f"Dataset/Pics_{observation_period}_{test_period}_{indicator}_{permno}_Torch/{subset}"
    
    if os.path.isdir(pic_path) == True:
        pass
    else:
        os.makedirs(pic_path)

    pic_path1 = f"Dataset/Pics_{observation_period}_{test_period}_{indicator}_{permno}_Torch/{subset}"
    
    if os.path.isdir(pic_path1) == True:
        pass
    else:
        os.makedirs(pic_path1)
    
    labels = pd.DataFrame(columns = ['Date', 'Permno', 'Ret_5d', 'Ret_20d', 'Ret_60d'])
    image_array = np.array()
    
    for i, tqdm in zip(range(iterations), tqdm(range(iterations))):
        
        signal = None
        ret = None
        
        start = i * test_period
        end = observation_period + i * test_period
        lookback = data.iloc[start : end, :]
        lookback_class5 = data.iloc[end : end + 5]
        lookback_class20 = data.iloc[end : end + 20]
        lookback_class60 = data.iloc[end : end + 60]

#         print(lookback_class60)
        
#         try:
        openprc5 = lookback_class5.iloc[0, 0]
        closeprc5 = lookback_class5.iloc[-1, 3]
        openprc20 = lookback_class20.iloc[0, 0]
        closeprc20 = lookback_class20.iloc[-1, 3]
        openprc60 = lookback_class60.iloc[0, 0]
        closeprc60 = lookback_class60.iloc[-1, 3]
        ret5 = (closeprc5 - openprc5)/openprc5
        ret20 = (closeprc20 - openprc20)/openprc20
        ret60 = (closeprc60 - openprc60)/openprc60
        
        labels.loc[lookback_class5.index[0], :] = [lookback_class5.index[0], permno, ret5, ret20, ret60]
        
#         except:
#             ret = (int(lookback_class.iloc[-1, :-3].mean()) - int(lookback_class.iloc[0, :-2].mean()))/int(lookback_class.iloc[0, :-2].mean())

        # buy signal -- 1
        # sell signal -- 0
            
        graph = img_graph(lookback, height = height)
        graph.save(f"Dataset/Pics_{observation_period}_{test_period}_{indicator}_{permno}_Torch/{subset}/{a_count}.jpeg")
        a_count += 1

    labels.reset_index(inplace = True, drop = True)
    labels_df = pd.concat([labels_df, labels], axis = 0, ignore_index = True)
    labels_df.to_csv(f"C:/Users/hayk.darbinyan/codes/my_Venvs/_venv-310-jupyter/Thesis/Dataset/Pics_{observation_period}_{test_period}_{indicator}_{permno}_Torch/Labels_{subset}.csv", index = True)
    pbar.update(k + 1)    
    
pbar.finish()

  0%|          | 0/159995 [00:00<?, ?it/s]

100% |########################################################################|


In [14]:
labels_df.to_csv(f"C:/Users/hayk.darbinyan/codes/my_Venvs/_venv-310-jupyter/Thesis/Dataset/Pics_{observation_period}_{test_period}_{indicator}_Torch/Labels_{subset}.csv", index = True)

In [45]:
ex = meta.loc["10057"].set_index('date')
ex

,BIDLO,ASKHI,PRC,VOL,RET,BID,ASK,OPENPRC
date,,,,,,,,
1994-01-03,9.375,9.625,9.625,3000.0,0.013158,9.250,9.625,9.500
1994-01-04,9.500,10.000,10.000,12500.0,0.038961,9.750,10.250,9.500
1994-01-06,9.750,9.875,9.875,8200.0,0.0,9.625,10.125,9.875
1994-01-10,9.625,9.875,9.750,10000.0,-0.012658,9.750,10.250,9.625
1994-01-11,9.875,9.875,9.875,10800.0,0.012821,9.750,10.125,9.875
...,...,...,...,...,...,...,...,...
1996-06-26,29.875,30.000,29.875,17200.0,-0.004167,29.875,30.125,29.875
1996-06-27,29.875,30.000,30.000,20300.0,0.004184,29.750,30.125,29.875
1996-06-28,29.875,30.000,30.000,19800.0,0.0,29.750,30.125,29.875


In [198]:
observation_period = 60
height = 96
test_period = 30
subset = 'Test_2008'
iters = set(meta.index)
indicator = 'MFI'
a_count = 0
labels_df = pd.DataFrame(columns = ['Date', 'Permno', 'Ret_5d', 'Ret_20d', 'Ret_60d'])

pbar = progressbar.ProgressBar(maxval = len(iters)).start()

for k, permno in enumerate(list(iters)):
    
    data = meta.loc[permno, :]
    if data.iloc[0, 0] == '1994-01-03' and data.iloc[-1, 0] == '2016-12-30': # 1994-01-03, 2016-12-30
        pass
    else:
        continue
    start = '2009-01-03'
    end = '2012-12-30'
    data = data.loc[:, ['date', 'OPENPRC', 'ASKHI', 'BIDLO', 'PRC', 'VOL']]
    data['MA'] = data['PRC'].rolling(observation_period).mean()
    data['VOL'] = talib.MFI(data['ASKHI'], data['BIDLO'], data['PRC'], data['VOL'], timeperiod = observation_period)
    data = data.iloc[observation_period + 1:, :]
    data['ret_5d'] = np.zeros(len(data))
    data['ret_20d'] = np.zeros(len(data))
    data['ret_60d'] = np.zeros(len(data))
    data.set_index('date', inplace = True)
    data = data.loc[start:end, :]
    l = len(data)
    iterations = round(l/(test_period)) - 6
    
    pic_path = f"Dataset/Pics_{observation_period}_{test_period}_{indicator}_Torch/{subset}"
    
    if os.path.isdir(pic_path) == True:
        pass
    else:
        os.makedirs(pic_path)

    pic_path1 = f"Dataset/Pics_{observation_period}_{test_period}_{indicator}_Torch/{subset}"
    
    if os.path.isdir(pic_path1) == True:
        pass
    else:
        os.makedirs(pic_path1)
    
    labels = pd.DataFrame(columns = ['Date', 'Permno', 'Ret_5d', 'Ret_20d', 'Ret_60d'])
    
    for i in range(iterations):
        
        signal = None
        ret = None
        
        start = i * test_period
        end = observation_period + i * test_period
        lookback = data.iloc[start : end, :]
        lookback_class5 = data.iloc[end : end + 5]
        lookback_class20 = data.iloc[end : end + 20]
        lookback_class60 = data.iloc[end : end + 60]

#         print(lookback_class60)
        
#         try:
        openprc5 = lookback_class5.iloc[0, 0]
        closeprc5 = lookback_class5.iloc[-1, 3]
        openprc20 = lookback_class20.iloc[0, 0]
        closeprc20 = lookback_class20.iloc[-1, 3]
        openprc60 = lookback_class60.iloc[0, 0]
        closeprc60 = lookback_class60.iloc[-1, 3]
        ret5 = (closeprc5 - openprc5)/openprc5
        ret20 = (closeprc20 - openprc20)/openprc20
        ret60 = (closeprc60 - openprc60)/openprc60
        
        labels.loc[lookback_class5.index[0], :] = [lookback_class5.index[0], permno, ret5, ret20, ret60]
        
#         except:
#             ret = (int(lookback_class.iloc[-1, :-3].mean()) - int(lookback_class.iloc[0, :-2].mean()))/int(lookback_class.iloc[0, :-2].mean())

        # buy signal -- 1
        # sell signal -- 0
            
        graph = img_graph(lookback, height = height)
        graph.save(f"Dataset/Pics_{observation_period}_{test_period}_{indicator}_Torch/{subset}/{a_count}.jpeg")
        a_count += 1

    labels.reset_index(inplace = True, drop = True)
    labels_df = pd.concat([labels_df, labels], axis = 0, ignore_index = True)
    pbar.update(k + 1)    
    
pbar.finish()

100% |########################################################################|


In [199]:
labels_df.to_csv(f"C:/Users/hayk.darbinyan/codes/my_Venvs/_venv-310-jupyter/Thesis/Dataset/Pics_{observation_period}_{test_period}_{indicator}_Torch/Labels_{subset}.csv", index = True)

In [18]:
observation_period = 5
height = 32
test_period = 3
subset = 'Train_2016'
iters = set(meta.index)
indicator = 'MFI'
a_count = 0
labels_df = pd.DataFrame(columns = ['Date', 'Permno', 'Ret_5d', 'Ret_20d', 'Ret_60d'])

pbar = progressbar.ProgressBar(maxval = len(iters)).start()

for k, permno in enumerate(list(iters)):
    
    data = meta.loc[permno, :]
    if data.iloc[0, 0] == '1994-01-03' and data.iloc[-1, 0] == '2016-12-30': # 1994-01-03, 2016-12-30
        pass
    else:
        continue
    start = '2010-01-03'
    end = '2016-12-30'
    data = data.loc[:, ['date', 'OPENPRC', 'ASKHI', 'BIDLO', 'PRC', 'VOL']]
    data['MA'] = data['PRC'].rolling(observation_period).mean()
    data['VOL'] = talib.MFI(data['ASKHI'], data['BIDLO'], data['PRC'], data['VOL'], timeperiod = observation_period)
    data = data.iloc[observation_period + 1:, :]
    data['ret_5d'] = np.zeros(len(data))
    data['ret_20d'] = np.zeros(len(data))
    data['ret_60d'] = np.zeros(len(data))
    data.set_index('date', inplace = True)
    data = data.loc[start:end, :]
    l = len(data)
    iterations = round(l/(test_period)) - 6
    
    pic_path = f"Dataset/Pics_{observation_period}_{test_period}_{indicator}_Torch/{subset}"
    
    if os.path.isdir(pic_path) == True:
        pass
    else:
        os.makedirs(pic_path)

    pic_path1 = f"Dataset/Pics_{observation_period}_{test_period}_{indicator}_Torch/{subset}"
    
    if os.path.isdir(pic_path1) == True:
        pass
    else:
        os.makedirs(pic_path1)
    
    labels = pd.DataFrame(columns = ['Date', 'Permno', 'Ret_5d', 'Ret_20d', 'Ret_60d'])
    
    for i in range(iterations):
        
        signal = None
        ret = None
        
        start = i * test_period
        end = observation_period + i * test_period
        lookback = data.iloc[start : end, :]
        lookback_class5 = data.iloc[end : end + 5]
        lookback_class20 = data.iloc[end : end + 20]
        lookback_class60 = data.iloc[end : end + 60]

#         print(lookback_class60)
        
#         try:
        openprc5 = lookback_class5.iloc[0, 0]
        closeprc5 = lookback_class5.iloc[-1, 3]
        openprc20 = lookback_class20.iloc[0, 0]
        closeprc20 = lookback_class20.iloc[-1, 3]
        openprc60 = lookback_class60.iloc[0, 0]
        closeprc60 = lookback_class60.iloc[-1, 3]
        ret5 = (closeprc5 - openprc5)/openprc5
        ret20 = (closeprc20 - openprc20)/openprc20
        ret60 = (closeprc60 - openprc60)/openprc60
        
        labels.loc[lookback_class5.index[0], :] = [lookback_class5.index[0], permno, ret5, ret20, ret60]
        
#         except:
#             ret = (int(lookback_class.iloc[-1, :-3].mean()) - int(lookback_class.iloc[0, :-2].mean()))/int(lookback_class.iloc[0, :-2].mean())

        # buy signal -- 1
        # sell signal -- 0
            
        graph = img_graph(lookback, height = height)
        graph.save(f"Dataset/Pics_{observation_period}_{test_period}_{indicator}_Torch/{subset}/{a_count}.jpeg")
        a_count += 1

    labels.reset_index(inplace = True, drop = True)
    labels_df = pd.concat([labels_df, labels], axis = 0, ignore_index = True)
    pbar.update(k + 1)    
    
pbar.finish()

100% |########################################################################|


In [19]:
labels_df.to_csv(f"C:/Users/hayk.darbinyan/codes/my_Venvs/_venv-310-jupyter/Thesis/Dataset/Pics_{observation_period}_{test_period}_{indicator}_Torch/Labels_{subset}.csv", index = True)

In [202]:
meta = pd.read_csv('Test.csv', index_col = 0)
print(len(meta))
meta.dropna(axis = 0, inplace = True)
print(len(meta))
meta.head()

776333
771866


,date,BIDLO,ASKHI,PRC,VOL,BID,ASK,OPENPRC
PERMNO,,,,,,,,
10104,2017-01-03,38.300,38.69,38.55,11051341.0,38.53,38.54,38.45
10104,2017-01-04,38.550,38.92,38.74,9545541.0,38.74,38.75,38.55
10104,2017-01-05,38.405,38.95,38.64,12064718.0,38.64,38.65,38.67
10104,2017-01-06,38.380,38.75,38.45,14829725.0,38.45,38.46,38.75
10104,2017-01-09,38.470,39.45,39.03,15587888.0,39.04,39.05,38.53


In [203]:
observation_period = 60
height = 96
test_period = 30
subset = 'Test'
iters = set(meta.index)
indicator = 'MFI'
a_count = 0
labels_df = pd.DataFrame(columns = ['Date', 'Permno', 'Ret_5d', 'Ret_20d', 'Ret_60d'])

pbar = progressbar.ProgressBar(maxval = len(iters)).start()

for k, permno in enumerate(list(iters)):
    
    data = meta.loc[permno, :]
    if data.iloc[0, 0] == '2017-01-03' and data.iloc[-1, 0] == '2022-12-30': # 1994-01-03, 2016-12-30
        pass
    else:
        continue
    start = '2000-01-03'
    end = '2008-12-30'
    data = data.loc[:, ['date', 'OPENPRC', 'ASKHI', 'BIDLO', 'PRC', 'VOL']]
    data['MA'] = data['PRC'].rolling(observation_period).mean()
    data['VOL'] = talib.MFI(data['ASKHI'], data['BIDLO'], data['PRC'], data['VOL'], timeperiod = observation_period)
    data = data.iloc[observation_period + 1:, :]
    data['ret_5d'] = np.zeros(len(data))
    data['ret_20d'] = np.zeros(len(data))
    data['ret_60d'] = np.zeros(len(data))
    data.set_index('date', inplace = True)
#     data = data.loc[start:end, :]
    l = len(data)
    iterations = round(l/(test_period)) - 6
    
    pic_path = f"Dataset/Pics_{observation_period}_{test_period}_{indicator}_Torch/{subset}"
    
    if os.path.isdir(pic_path) == True:
        pass
    else:
        os.makedirs(pic_path)

    pic_path1 = f"Dataset/Pics_{observation_period}_{test_period}_{indicator}_Torch/{subset}"
    
    if os.path.isdir(pic_path1) == True:
        pass
    else:
        os.makedirs(pic_path1)
    
    labels = pd.DataFrame(columns = ['Date', 'Permno', 'Ret_5d', 'Ret_20d', 'Ret_60d'])
    
    for i in range(iterations):
        
        signal = None
        ret = None
        
        start = i * test_period
        end = observation_period + i * test_period
        lookback = data.iloc[start : end, :]
        lookback_class5 = data.iloc[end : end + 5]
        lookback_class20 = data.iloc[end : end + 20]
        lookback_class60 = data.iloc[end : end + 60]

#         print(lookback_class60)
        
#         try:
        openprc5 = lookback_class5.iloc[0, 0]
        closeprc5 = lookback_class5.iloc[-1, 3]
        openprc20 = lookback_class20.iloc[0, 0]
        closeprc20 = lookback_class20.iloc[-1, 3]
        openprc60 = lookback_class60.iloc[0, 0]
        closeprc60 = lookback_class60.iloc[-1, 3]
        ret5 = (closeprc5 - openprc5)/openprc5
        ret20 = (closeprc20 - openprc20)/openprc20
        ret60 = (closeprc60 - openprc60)/openprc60
        
        labels.loc[lookback_class5.index[0], :] = [lookback_class5.index[0], permno, ret5, ret20, ret60]
        
#         except:
#             ret = (int(lookback_class.iloc[-1, :-3].mean()) - int(lookback_class.iloc[0, :-2].mean()))/int(lookback_class.iloc[0, :-2].mean())

        # buy signal -- 1
        # sell signal -- 0
            
        graph = img_graph(lookback, height = height)
        graph.save(f"Dataset/Pics_{observation_period}_{test_period}_{indicator}_Torch/{subset}/{a_count}.jpeg")
        a_count += 1

    labels.reset_index(inplace = True, drop = True)
    labels_df = pd.concat([labels_df, labels], axis = 0, ignore_index = True)
    pbar.update(k + 1)    
    
pbar.finish()

100% |########################################################################|


In [204]:
labels_df.to_csv(f"C:/Users/hayk.darbinyan/codes/my_Venvs/_venv-310-jupyter/Thesis/Dataset/Pics_{observation_period}_{test_period}_{indicator}_Torch/Labels_{subset}.csv", index = True)

In [176]:
data['UP'] = talib.WCLPRICE(data['ASKHI'],data['BIDLO'],data['PRC'])
data.head(25)

,OPENPRC,ASKHI,BIDLO,PRC,VOL,MA,ret_5d,ret_20d,ret_60d,UP
date,,,,,,,,,,
2000-01-03,23.8750,24.00000,23.0625,23.9375,48.324738,23.175000,0.0,0.0,0.0,23.734375
2000-01-04,23.6250,24.18750,23.1250,23.8750,47.839305,23.131250,0.0,0.0,0.0,23.765625
2000-01-05,23.6250,24.43750,23.6250,24.3125,51.434030,23.090625,0.0,0.0,0.0,24.171875
2000-01-06,24.3750,25.37500,24.3750,25.1875,57.587530,23.153125,0.0,0.0,0.0,25.031250
2000-01-07,25.0625,25.75000,25.0000,25.7500,60.936790,23.278125,0.0,0.0,0.0,25.562500
2000-01-10,25.6875,26.00000,25.4375,25.8750,61.645287,23.434375,0.0,0.0,0.0,25.796875
2000-01-11,25.8750,25.93750,25.4375,25.5625,58.837028,23.587500,0.0,0.0,0.0,25.625000
2000-01-12,25.7500,26.00000,25.5625,25.8125,60.357791,23.768750,0.0,0.0,0.0,25.796875
2000-01-13,25.8125,26.12500,25.6875,26.0000,61.481269,23.971875,0.0,0.0,0.0,25.953125


In [126]:
data.head(100)

,OPENPRC,ASKHI,BIDLO,PRC,VOL,SAR,ret_5d,ret_20d,ret_60d,OBV,OBVEMA
date,,,,,,,,,,,
2017-03-31,74.64,74.870,74.44,74.55,1627623.0,75.631207,0.0,0.0,0.0,1627623.0,NaN
2017-04-03,74.67,74.870,73.06,73.67,2065557.0,75.481735,0.0,0.0,0.0,-437934.0,NaN
2017-04-04,73.61,73.860,73.10,73.45,1873175.0,75.287996,0.0,0.0,0.0,-2311109.0,NaN
2017-04-05,73.83,74.360,72.73,72.86,1537784.0,75.109756,0.0,0.0,0.0,-3848893.0,NaN
2017-04-06,72.92,73.520,72.51,73.11,1260185.0,74.871781,0.0,0.0,0.0,-2588708.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2017-08-16,79.34,79.770,79.26,79.46,748021.0,82.174183,0.0,0.0,0.0,3489378.0,3.919904e+06
2017-08-17,79.04,79.260,77.58,77.61,1106530.0,81.874648,0.0,0.0,0.0,2382848.0,3.612493e+06
2017-08-18,77.45,78.485,77.15,77.71,1407046.0,81.445183,0.0,0.0,0.0,3789894.0,3.647973e+06
